In [14]:
import pandas as pd
import os
import shutil
from sqlalchemy import create_engine, text
from sshtunnel import SSHTunnelForwarder
from sqlalchemy.orm import sessionmaker

# Пути к папкам ввода и вывода
folder_path_in = 'C:\\Users\\dmandree\\Downloads\\TL_new'
folder_path_out = 'C:\\Users\\dmandree\\Downloads\\TL_arch'

# Получение списка файлов в папке
file_list = os.listdir(folder_path_in)

# Создание пустого DataFrame
dfs = []

# Чтение всех файлов сразу и объединение их в один DataFrame
for file in file_list:
    file_path = os.path.join(folder_path_in, file)
    data = pd.read_excel(file_path, sheet_name='TurnoverList')
    dfs.append(data)
    # Перемещение обработанного файла в папку вывода
    new_path = os.path.join(folder_path_out, file)
    if os.path.exists(new_path):
        try:
            os.remove(new_path)
        except PermissionError:
            print("Файл используется другим процессом и не может быть удален.")
    try:
        shutil.move(file_path, folder_path_out)
    except (IOError, shutil.Error) as e:
        print(f"Error while moving file '{file_path}': {e}")
    
df = pd.concat(dfs)
    
# Преобразование столбца "Day" к типу даты
df["Day"] = pd.to_datetime(df["Day"]).dt.date

# Фильтрация данных по определенным компаниям
df = df[df['Company'].isin(['Guess Kazakhstan', 'Guess CIS'])]

# Приведение названий столбцов к нижнему регистру
df.columns = df.columns.str.lower()

# Создание списка уникальных дат
unique_combinations = df['day'].unique()

# Создание DataFrame для уникальных комбинаций
outer_df = pd.DataFrame(unique_combinations, columns=['key'])

# Конфигурация SSH туннеля
ssh_tunnel = SSHTunnelForwarder(
    ('79.174.86.163', 22),
    ssh_username='root',
    ssh_password='S0SJcmYwL0ZsmUId',
    remote_bind_address=('127.0.0.1', 5432),
    local_bind_address=('127.0.0.1', 8001)
)

# Запуск SSH туннеля
with ssh_tunnel: 

    # Конфигурация подключения к базе данных
    params = {
        'database': 'postgres',
        'user': 'postgres',
        'password': '1296',
        'host': 'localhost',
        'port': ssh_tunnel.local_bind_port
    }

    # Создание SQLAlchemy engine
    engine_str = f"postgresql://{params['user']}:{params['password']}@{params['host']}:{params['port']}/{params['database']}"
    engine = create_engine(engine_str)

    # Создание сессии
    Session = sessionmaker(bind=engine)
    
    with Session() as session:
        # Выполнение запроса и получение результатов
        query = text('select DISTINCT day as key from sales')
        result = session.execute(query).fetchall()

        # Чтение результатов запроса в DataFrame
        inner_df = pd.read_sql(query, engine)

        # Объединение DataFrame по столбцу 'key'
        intersection_df = pd.merge(outer_df, inner_df, on='key', how='inner')['key'].tolist()

        # Удаление пересечений в базе данных      
        if intersection_df:
            delete_query = text('DELETE FROM sales WHERE day = ANY(:keys)')
            try:
                session.execute(delete_query, {'keys': intersection_df})
            except Exception as e:
                print(f"Error while deleting records: {e}")
            finally:
                session.commit()

        # Загрузка данных в базу данных
        try:
            df.to_sql("sales", engine, if_exists="append", index=False)
            session.close()
        except Exception as e:
            print(f"Error while appending records: {e}")



